***Connect to google drive colab***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Installation of dependencies***

In [ ]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


***Display audio***

In [ ]:
from IPython.display import Audio

# Function to handle audio playback
def play_audio(file_path, rate=None):
    """Plays an audio file.

    Args:
        file_path (str): The path to the audio file.
        rate (int, optional): The sample rate of the audio.
                               If None, attempts to autodetect.
                               Defaults to None.
    """
    # If rate is not provided, assume it's an audio file and let Audio autodetect
    # If rate is provided, assume data is numpy array and pass rate
    audio = Audio(filename=file_path, rate=44100)
    display(audio)

audio_file = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals2.mp3"
play_audio(audio_file)

Output hidden; open in https://colab.research.google.com to view.

***Speech recognation***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals2.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the completed transcript object
result=transcript['text']
print(result)

Hi, Mr. Roberts. I'm here to check your vitals. How are you feeling this morning? I'm okay. Just a little sore from yesterday. I understand. I'll be as gentle as I can. Let's start with your blood pressure. Could you hold out your arm for me? Sure. Okay. Just relax your arm here. This might feel a little tough. Alright. Your blood pressure is 118 76, which looks good. Now I'll check your pulse. I'll be placing my fingers on your wrist, okay? Yes, go ahead. Your pulse is 68 beats per minute. Nice and steady. Next, I'll take your temperature. Could you open your mouth slightly so I can place this under your tongue? Thank you. Your temperature is 98.4 degrees Fahrenheit. So that's normal. Lastly, I'm going to check your oxygen saturation. I clip this device to your fingertip. Just hold still. Alright. Your oxygen level is 97 degrees. Next. 97%, which is good. Everything is looking great. If you're uncomfortable or need anything, please let us know. Thank you. I'll let you know if I feel a

In [ ]:
print(transcript)

{'id': '13dc8648-0032-4f94-aaba-5331f140def9', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/697e36d1-7191-4345-ac2b-d933325bafae', 'text': "Hi, Mr. Roberts. I'm here to check your vitals. How are you feeling this morning? I'm okay. Just a little sore from yesterday. I understand. I'll be as gentle as I can. Let's start with your blood pressure. Could you hold out your arm for me? Sure. Okay. Just relax your arm here. This might feel a little tough. Alright. Your blood pressure is 118 76, which looks good. Now I'll check your pulse. I'll be placing my fingers on your wrist, okay? Yes, go ahead. Your pulse is 68 beats per minute. Nice and steady. Next, I'll take your temperature. Could you open your mouth slightly so I can place this under your tongue? Thank you. Your temperature is 98.4 degrees Fahrenheit. So that's normal. Lastly, I'm going to check your o

'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397'

In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [transcript['text'][i:i+110] for i in range(0, len(transcript['text']), 110)]

# Join the chunks with line breaks
formatted_text = "\n".join(chunks)

print(formatted_text)

Hi, Mr. Roberts. I'm here to check your vitals. How are you feeling this morning? I'm okay. Just a little sore
 from yesterday. I understand. I'll be as gentle as I can. Let's start with your blood pressure. Could you hol
d out your arm for me? Sure. Okay. Just relax your arm here. This might feel a little tough. Alright. Your blo
od pressure is 118 76, which looks good. Now I'll check your pulse. I'll be placing my fingers on your wrist, 
okay? Yes, go ahead. Your pulse is 68 beats per minute. Nice and steady. Next, I'll take your temperature. Cou
ld you open your mouth slightly so I can place this under your tongue? Thank you. Your temperature is 98.4 deg
rees Fahrenheit. So that's normal. Lastly, I'm going to check your oxygen saturation. I clip this device to yo
ur fingertip. Just hold still. Alright. Your oxygen level is 97 degrees. Next. 97%, which is good. Everything 
is looking great. If you're uncomfortable or need anything, please let us know. Thank you. I'll let you know i
f

***Speaker Diarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "speaker_labels": True
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals2.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcrSpeaker = create_transcript(your_api_token, upload_url)

# Iterate through the utterances and print speaker information
print("\nSpeakers' Information:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    print(f"Speaker {speaker}: {text}\nConfidence: {confidence}\n")


Speakers' Information:


Speaker A: Hi, Mr. Roberts. I'm here to check your vitals. How are you feeling this morning? I'm okay. Just a little sore from yesterday. I understand. I'll be as gentle as I can. Let's start with your blood pressure. Could you hold out your arm for me? Sure. Okay. Just relax your arm here. This might feel a little tough. Alright. Your blood pressure is 118 76, which looks good. Now I'll check your pulse. I'll be placing my fingers on your wrist, okay? Yes, go ahead. Your pulse is 68 beats per minute. Nice and steady. Next, I'll take your temperature. Could you open your mouth slightly so I can place this under your tongue? Thank you. Your temperature is 98.4 degrees Fahrenheit. So that's normal. Lastly, I'm going to check your oxygen saturation. I clip this device to your fingertip. Just hold still. Alright. Your oxygen level is 97 degrees. Next. 97%, which is good. Everything is looking great. If you're uncomfortable or need anything, please let us know. Tha

In [ ]:
print(transcrSpeaker['utterances'])

[{'speaker': 'A', 'text': "Hi, Mr. Roberts. I'm here to check your vitals. How are you feeling this morning? I'm okay. Just a little sore from yesterday. I understand. I'll be as gentle as I can. Let's start with your blood pressure. Could you hold out your arm for me? Sure. Okay. Just relax your arm here. This might feel a little tough. Alright. Your blood pressure is 118 76, which looks good. Now I'll check your pulse. I'll be placing my fingers on your wrist, okay? Yes, go ahead. Your pulse is 68 beats per minute. Nice and steady. Next, I'll take your temperature. Could you open your mouth slightly so I can place this under your tongue? Thank you. Your temperature is 98.4 degrees Fahrenheit. So that's normal. Lastly, I'm going to check your oxygen saturation. I clip this device to your fingertip. Just hold still. Alright. Your oxygen level is 97 degrees. Next. 97%, which is good. Everything is looking great. If you're uncomfortable or need anything, please let us know. Thank you. I'

***Summarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "summarization": True,
        "summary_model": "informative",
        "summary_type": "bullets"
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /vitals2.mp3"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the summary
print(transcript["summary"])

- Hi, Mr. Roberts. How are you feeling this morning? I'm okay. Just a little sore from yesterday. Let's start with your blood pressure. Next, I'll take your temperature. Lastly, I'm going to check your oxygen saturation. Everything is looking great.


***Translate the text from english to any language the user want***

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='fr')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [result[i:i+chunk_size] for i in range(0, len(result), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



Bonjour, M. Roberts. Je suis ici pour vérifier vos signes vitaux. Comment vous sentez-vous ce matin ? Je vais bien. Juste un peu endolori depuis hier. Je comprends. Je serai aussi doux que possible. Commençons par votre tension artérielle. Pourriez-vous me tendre le bras ? Bien sûr. D'accord. Détendez simplement votre bras ici. Cela peut sembler un peu difficile. Très bien. Votre tension artérielle est de 118 76, ce qui semble bon. Maintenant, je vais vérifier votre pouls. Je vais placer mes doigts sur votre poignet, d'accord ? Oui, allez-y. Votre pouls est de 68 battements par minute. Bien et stable. Ensuite, je vais prendre votre température. Pourriez-vous ouvrir légèrement la bouche pour que je puisse placer cela sous votre langue ? Merci. Votre température est de 98,4 degrés Fahrenheit. C'est donc normal. Enfin, je vais vérifier votre saturation en oxygène. J'attache cet appareil au bout de votre doigt. Restez immobile. Très bien. Votre taux d'oxygène est de 97 degrés. Ensuite. 97 

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [transcript["summary"][i:i+chunk_size] for i in range(0, len(transcript["summary"]), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



- Hallo, Mr. Roberts. Wie fühlen Sie sich heute Morgen? Mir geht es gut. Nur ein bisschen Schmerzen von gestern. Beginnen wir mit Ihrem Blutdruck. Als Nächstes messe ich Ihre Temperatur. Zum Schluss prüfe ich Ihre Sauerstoffsättigung. Alles sieht gut aus.


In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [translated_text[i:i+110] for i in range(0, len(translated_text), 110)]

# Join the chunks with line breaks
formatted_text_Trans= "\n".join(chunks)

print(formatted_text_Trans)

- Hallo, Mr. Roberts. Wie fühlen Sie sich heute Morgen? Mir geht es gut. Nur ein bisschen Schmerzen von gester
n. Beginnen wir mit Ihrem Blutdruck. Als Nächstes messe ich Ihre Temperatur. Zum Schluss prüfe ich Ihre Sauers
toffsättigung. Alles sieht gut aus.


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='mr')

# Iterate through the utterances and print translated speaker information
print("\nInformation des intervenants:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    # Translate the speaker and text
    translated_speaker = translator.translate(speaker)
    translated_text = translator.translate(text)

    print(f"Intervenant {translated_speaker}: {translated_text}\nConfiance: {confidence}\n")



Information des intervenants:


Intervenant ए: हाय, मिस्टर रॉबर्ट्स. मी तुमच्या जीवनावश्यक गोष्टी तपासण्यासाठी आलो आहे. आज सकाळी तुम्हाला कसे वाटते? मी ठीक आहे. कालपासून जरा दुखत आहे. मला समजते. मी शक्य तितके सौम्य राहीन. चला तुमच्या रक्तदाबापासून सुरुवात करूया. तू माझ्यासाठी तुझा हात धरू शकशील का? नक्की. ठीक आहे. इथे फक्त तुमचा हात आराम करा. हे थोडे कठीण वाटू शकते. ठीक आहे. तुमचा रक्तदाब 118 76 आहे, जो चांगला दिसतो. आता मी तुमची नाडी तपासतो. मी तुझ्या मनगटावर बोटं ठेवतो, ठीक आहे? होय, पुढे जा. तुमची नाडी 68 बीट्स प्रति मिनिट आहे. छान आणि स्थिर. पुढे, मी तुमचे तापमान घेईन. मी हे तुमच्या जिभेखाली ठेवू शकेन म्हणून तुम्ही तुमचे तोंड थोडेसे उघडू शकता का? धन्यवाद. तुमचे तापमान ९८.४ अंश फॅरेनहाइट आहे. तर ते सामान्य आहे. शेवटी, मी तुमचे ऑक्सिजन संपृक्तता तपासणार आहे. मी हे उपकरण तुमच्या बोटाच्या टोकावर क्लिप करतो. जरा थांबा. ठीक आहे. तुमची ऑक्सिजन पातळी 97 अंश आहे. पुढे. 97%, जे चांगले आहे. सर्व काही छान दिसत आहे. तुम्हाला अस्वस्थ वाटत असल्यास किंवा कशाची गरज असल्यास, कृपया आम्हाला कळवा. धन्